## Async programming
Task asynchronous programming model(TAP).
Task await doesnt block thread.
if any part of code is async entire code block is async- tells compiler that some threads will be released when await. 

In depth: https://docs.microsoft.com/en-us/dotnet/standard/async-in-depth

Important!
when task is I/O bound use async await in standard form: if I/O-bound task is run no thread is dedicated to running the task.
Most of the time application hands off work to other system lib that takes time. thats why servers should use async- it frees 
server pool when I/O task is being done. IT should be done like that is asp net too:
https://stackoverflow.com/questions/41953102/using-async-await-or-task-in-web-api-controller-net-core

In [ ]:
//this starts task 
private bool CalculateDamageDone()
{
    // Code omitted:
    //
    // Does an expensive calculation and returns
    // the result of that calculation.
    return true;
}

var task  = async (o, e) =>
{
    // This line will yield control to the UI while CalculateDamageDone()
    // performs its work. The UI thread is free to perform other work.
    var damageResult = await Task.Run(() => CalculateDamageDone());
    // DisplayDamage(damageResult);
};

This is because it spawns another thread. Keep in mind this does not offer any extra protection for shared data.
You should use Task.Run, but not within any code you want to be reusable (i.e., library code). So you use Task.Run to call the method, not as part of the implementation of the method.
Async void should only be used for event handlers. On ui exceptions are thrown onto dispatcher. Normaly they cant be awaited on
and exception crashes app.
Tread carefully when using async lambdas in LINQ expressions

returning reference type may impact performance. Consider using ValueTask from c#7.0. You may define your own return type with gGetAwaiter method to return task.
IAsyncEnumerable works for async methods that return streams from C#8.0

### Cancelling tasks

W celu przerywania taskow uzywa sie CancellationTokenSource. Podaje sie taki obiekt w parametrze do kazdeego taska.

### Obslugiwanie taskow w miare ich ukonczenia
metoda ta jest zlozona obliczeniowa w przypadku duzej ilosci taskow powinno sie uzywac metod opisanych w linku ponizej

In [ ]:


while (downloadTasks.Any())
    {
        Task<int> finishedTask = await Task.WhenAny(downloadTasks);
        downloadTasks.Remove(finishedTask);
        total += await finishedTask;
    }

This works for small number of tasks, for more see https://devblogs.microsoft.com/pfxteam/processing-tasks-as-they-complete/
since it creates N^2 algorithm for no reason.

# exceptions for tasks

Tasks that throws is called faulted.
Tasks hold exceptions thrown in Task.Exception property as System.AggregateException.
Exception is thrown when task is awaited on and first exception from AggregateException.InnerExceptions is thrown.

Task logic: 
```
await Task.WhenAll(eggsTask, baconTask, toastTask);
Task finishedTask = await Task.WhenAny(breakfastTasks);
```


# Atrybuty
Atrybuty moga byc dodawane w runtime za pomoca refleksji. Konwencja: nazwy atrybutów kończą się zawsze na Attribute,


In [ ]:
// przyklad uzycia atrybutow dla parametru
void MethodA([In][Out] ref double x) { }
void MethodB([Out][In] ref double x) { }
void MethodC([In, Out] ref double x) { }

Attribute target: target to entity ktore atrybut aplikuje sie do.
Aby jawnie okreslic cel atrybutu nalezy uzyc skladni

In [ ]:
[target : attribute-list]

Target value	Applies to

assembly	Entire assembly

module	Current assembly module

field	Field in a class or a struct

event	Event

method	Method or get and set property accessors

param	Method parameters or set property accessor parameters

property	Property

return	Return value of a method, property indexer, or get property accessor

type	Struct, class, interface, enum, or delegate

In [ ]:
// default: applies to method
[ValidatedContract]
int Method1() { return 0; }

// applies to method
[method: ValidatedContract]
int Method2() { return 0; }

// applies to parameter
int Method3([ValidatedContract] string contract) { return 0; }

// applies to return value
[return: ValidatedContract]
int Method4() { return 0; }

Popularne użycia atrybutów:
Marshalling interopów 
Określanie właściwosci COM (Component Object Model)
Dll importy
Okreslanie pol do serializacji

Definiowanie wlasnego atrybutu z refleksja do jego odczytania:

In [ ]:
// Multiuse attribute.  
[System.AttributeUsage(System.AttributeTargets.Class |  
                       System.AttributeTargets.Struct,  
                       AllowMultiple = true)  // Multiuse attribute.  
]  
public class Author : System.Attribute  
{  
    string name;  
    public double version;  
  
    public Author(string name)  
    {  
        this.name = name;  
  
        // Default value.  
        version = 1.0;  
    }  
  
    public string GetName()  
    {  
        return name;  
    }  
}  
  
// Class with the Author attribute.  
[Author("P. Ackerman")]  
public class FirstClass  
{  
    // ...  
}  
  
// Class without the Author attribute.  
public class SecondClass  
{  
    // ...  
}  
  
// Class with multiple Author attributes.  
[Author("P. Ackerman"), Author("R. Koch", version = 2.0)]  
public class ThirdClass  
{  
    // ...  
}  
  
class TestAuthorAttribute  
{  
    static void Test()  
    {  
        PrintAuthorInfo(typeof(FirstClass));  
        PrintAuthorInfo(typeof(SecondClass));  
        PrintAuthorInfo(typeof(ThirdClass));  
    }  
  
    private static void PrintAuthorInfo(System.Type t)  
    {  
        System.Console.WriteLine("Author information for {0}", t);  
  
        // Using reflection.  
        System.Attribute[] attrs = System.Attribute.GetCustomAttributes(t);  // Reflection.  
  
        // Displaying output.  
        foreach (System.Attribute attr in attrs)  
        {  
            if (attr is Author)  
            {  
                Author a = (Author)attr;  
                System.Console.WriteLine("   {0}, version {1:f}", a.GetName(), a.version);  
            }  
        }  
    }  
}  
/* Output:  
    Author information for FirstClass  
       P. Ackerman, version 1.00  
    Author information for SecondClass  
    Author information for ThirdClass  
       R. Koch, version 2.00  
       P. Ackerman, version 1.00  
*/

### collections
Są dwa generalne podejscia uzycie arraya lub kolekcji.
Arraya uzywamy gdy mamy do cyznienia ze stała ilością mocno typowanych obiektów.

Kolekcje sa bardziej elastyczne potrafia zmieniac swoj rozmiar dynamicznie. Dla kolekcji obiektow o jednym typie uzywamy System.Collections.Generic.

Do System.Collection.Generic zaliczamy 
- Dictionary<TKey, TValue>- key value pairs 
- List<T>- no lista no
- Queue<T>- FIFO(first in first out)
- SortedList<TKey, TValue>- kolekcja par key/value posegregowana wedlug implementacji IComparer<T>
- Stack<T>- last in first out (LIFO) 

System.Collections.Concurent classes- klasy dające watkowe bezpieczne kolekcje.  
Przykladowymi sa:
- BlockingCollection<T>,
- ConcurrentDictionary<TK,TV>, 
- ConcurrentStack<T>

System.Collections classes- kolekcje nie przechowujące typowanych obiektów- obiekty w nich są traktowane jako Object.
- ArrayList reprezentuje arraya ktorego wielkosc jest dynamicznie zwiekszana w razie potrzeby
- Hasktable reprezentuje kolekcje key/value ktora jest segregowana wedlug kodu hash klucza zmienne przechowywane są w bucketach
- Queue FIFO
- stack LIFO

Jezeli obiekt implementuje ICompare mozna jego liste segregowac.

Tworzenie wlasnej kolekcji zachodzi przez imlementacje IEnumerable. Wymaga to zaipmlementowania enumeratora i metody GetEnumerator w klasie koleksji. Enumerator ma current i move next do iteracji

### Iteratory
Iterator sluzy do customowego chodzenia po kolekcji. Zwraca on przez yield return kolejne elementy.
Uzywajac slowa kluczowego yield okreslamy ze metoda, operator lub accessor get jest iteratorem.

In [9]:
static void Main()
{
    foreach (int number in SomeNumbers())
    {
        Console.Write(number.ToString() + " ");
    }
    // Output: 3 5 8
    Console.ReadKey();
}

public static System.Collections.IEnumerable SomeNumbers()
{
    yield return 3;
    yield return 5;
    yield return 8;
}

3 5 8 

Unhandled exception: System.InvalidOperationException: Cannot read keys when either application does not have a console or when console input has been redirected. Try Console.Read.
   at System.ConsolePal.ReadKey(Boolean intercept)
   at System.Console.ReadKey()
   at Submission#11.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

# Covariance i Contrvariance
Covariance in(contrvariance)-parametr moze uzyc bardziej derived klasy, out(covariance)-returnowac moze baedziej derived klase.
Stack thread: https://stackoverflow.com/questions/3445631/still-confused-about-covariance-and-contravariance-in-out

Uwaga kowariancja dziala tylko dla typów referencyjnych

In [ ]:
// The interface is covariant.
ICovariant<Button> ibutton = new SampleImplementation<Button>();
ICovariant<Object> iobj = ibutton;

// The class is invariant.
SampleImplementation<Button> button = new SampleImplementation<Button>();
// The following statement generates a compiler error
// because classes are invariant.
// SampleImplementation<Object> obj = button;

Dla kolekcji jest to przydatne na pryzkład w wypadku porównania: tworzymy wlasny comparer
 IEnumerable<Employee> noduplicates =  
            employees.Distinct<Employee>(new PersonComparer());  <-- uwaga w parametrze do distinct da sie podawac


Strong and weak typing:
weak typing no strong definition strongly typed lang can be dynamically typed!
! Note that in dynamically typed languages, values have types, not variables. Type inference: statyczne typowanie ktore domysla sie kompilator jak w Haskellu

### Expression trees
expression tree to tree like data structure gdize kazdy node reprezenture ekspresje typu operacja binarna lub wywolanie funkcji

Expression trees are also used in the dynamic language runtime (DLR) to provide interoperability between dynamic languages and .NET and to enable compiler writers to emit expression trees instead of Microsoft intermediate language (MSIL)

Drzewa ekspresji da sie wytwarzac z linqa lub recznie przez System.Linq.Expressions
Ekspresje można tworzyć z jednolinijkowych(tylko) lambd:

In [ ]:
Expression<Func<int, bool>> lambda = num => num < 5;  

Ekspresji używa się do Querying based on runtime state. Przykładem tego jest po prostu uzywanie IQueryable tak jak normlanie zapytania na bazie.
IQueryable ma dwa komponenty:
- Expression - a language- and datasource-agnostic representation of the current query's components, in the form of an expression tree.
- Provider - instancja providera LINQ który zna zasady materializacji query w wartości

Expression  trees mozna renderowac przy pomocy DebugView(todo zroobic program z tym)


Expression trees offer a way to inspect some piece of code before it is actually compiled, at runtime.

### Expressions vs delegates vs anomylous functions:
https://stackoverflow.com/questions/17709006/anonymous-methods-vs-lambda-expression
W przypadku gdyfunkcja spodziewa sie delegatu lambda jest skrótem do delegate{ ... }, jednak w przypadku Expressions<>
lambda jest przebudowywana w expression tree w celu pozniejszej optymalizacji.
Delegat daje tylkko metode do zbudowania, expression daje informacje co sie dzieje w srodku do czego np linq moze sie podpiac i optymalizowac na SQL.


### examples:
Few are the scenario's where you can use expression tree.

1) Dynamic Rule Engine - Rather than hard-coding the rule logic in your code. You can build your business-rule logic at run-time.

2) Dynamic Linq Queries - There are various scenario where developer need to write hard-coded queries. So you can eliminate this using expression tree.

## LINQ

Query expressions can be compiled to expression trees or to delegates, depending on the type that the query is applied to. IEnumerable<T> queries are compiled to delegates. IQueryable and IQueryable<T> queries are compiled to expression trees. For more information, see Expression trees.

Kiedy fluent exppression  typu lsit.Select a kiedy query expressino typu from list select .. ??
gdy robimy lety i joiny! se stacka O:
Neither is better: they serve different needs. Query syntax comes into its own when you want to leverage multiple range variables. This happens in three situations
- When using the let keyword
- When you have multiple generators (from clauses)
- When doing joins

W przypadku uzywania ArrayList trzeba podac typ!
### Linq grouping

In [ ]:
// custQuery is an IEnumerable<IGrouping<string, Customer>>
var custQuery =
    from cust in customers
    group cust by cust.City into custGroup
    where custGroup.Count() > 2
    orderby custGroup.Key
    select custGroup;

There are three separate ADO.NET (polaczenie odb z baza) Language-Integrated Query (LINQ) technologies: LINQ to DataSet, LINQ to SQL, and LINQ to Entities. LINQ to DataSet provides richer, optimized querying over the DataSet, LINQ to SQL enables you to directly query SQL Server database schemas, and LINQ to Entities allows you to query an Entity Data Model.

## assemblies
assembly to podstawoowa jednostka organizacyjna w .necie. asseembly sa implementowany jako .exe albo .dll

Kazdy komputer ktory ma CLR posiada Global Assembly Cache ktory przetrzymuje assemblies miedzy aplikacjami. Normalnie utrzymujemy assemlbies osobno
i wyszukujemy po lokalnym directory, niekioedy jednak mozemy odwolywac sie do <-- chyba tylko dotnet framework>

Assemblies can be static or dynamic. Static assemblies are stored on disk in portable executable (PE) files. Static assemblies can include interfaces, classes, and resources like bitmaps, JPEG files, and other resource files. You can also create dynamic assemblies, which are run directly from memory and aren't saved to disk before execution. You can save dynamic assemblies to disk after they have executed.

## Serialization
Json serialization noormalna serializacja mozna przy pomocy atrybutow i opcji zmieniac zachowanie.
Binarna serializacja- do kodu binarnego super szxbyka, niebezpieczna.


# Statements expressions and operators
typy statementów:
normalne: wybor z ifami, ekspresje, deklarace, lapanie wyjatkow, jump -break goto continue, iteracyjne,
Ciekawe:
- checked- normalnie kompilator wykryje overflow gdy dodamy dwie stale liczby, ale w przypadku gdy dodamy zmienna nie wykryje tego, uzywamy wtedy checked(duzaliczba+liczbaPowodujacaOverflow). Checked moze tez byc w blocku{}. Unchecked powoduje nie sprawdzanie przy dodawaniuliczb czy overflow.
- yield return - gdy iterator do tego dojdzie zwraca co tam jest, ale 'miejsce' w kodzie jest zapamietywane i egzekucja jest od neigo wznawiana.
Yield break przerywa iteracje.
- fixed- zapobiega GC relokacje zmiennej. Mozliwe tylkow  unsafe. N a przyklad robimy se zmienna refer i uzywamy wskaznika do niej. Fixed zabezpiieczy zeby wskaznik na nia wskazywal cnie
- label dla goto
- empty statement- przydatne! po prostu  mozna dac ; 
- lock- blokuje dostęp do kodu tlyko jednemu wątkowi jednocześnie

In [ ]:
//empty statement
void ProcessMessages()
{
    while (ProcessMessage())
        ; // Statement needed here.
}


//lock

lock (x)
{
    // Your code...
}

//wychodzi to samo co tutaj

object __lockObj = x;
bool __lockWasTaken = false;
try
{
    System.Threading.Monitor.Enter(__lockObj, ref __lockWasTaken);
    // Your code...
}
finally
{
    if (__lockWasTaken) System.Threading.Monitor.Exit(__lockObj);
}

### Reference i value equality

reference equality sprawdzamy System.Object.ReferenceEquals(a,b) i sprawdza to czy referujemy sie do tej samej zmiennej.


In [ ]:
  bool areEqual = System.Object.ReferenceEquals(a, b);
        // False:

Value equality dziala dla typow ze zdefiniowana metoda Equals i sprawdza wartosc typu. dl atypow prymitywnych jest to pproste dla klas trzeba samemu zdefiniowac interppretacje danych. Uwaga przy definiowaniu Equals powinno sie zeimplementowac je wedlug zasad matematycznych rownosci. Warto rowniez pobawid sie z gethashcode aby te same obiekty mialy taki sam hash! Uwaga reference equals nie powinno byc uzywane do stringow!

Uwaga od c#9 mozna dostac latwe porownanie za free przez recordy !

## Cast
Castowanie zachodzi w 4 sposoy:
- implicit
- explicit przy pomocy (typ)zmienna
- User definied 
- przy pomocy klas helper 

In [ ]:
//wlasna definicja casta
public readonly struct Digit
{
    private readonly byte digit;
    public static implicit operator byte(Digit d) => d.digit;
    public static explicit operator Digit(byte b) => new Digit(b);
}

Is operator:
Is testuje eskpresje czy jest danego typu, a od C#7 rowniez czy podaza za wzorem jak w haskellu. Uwaga nie dziala do lambd!